In [1]:
import pandas as pd
import numpy as np

In [4]:
data=pd.read_json('loan_data.json')
data.head()

,Application_ID,Gender,Married,Dependents,Education,Self_Employed,Credit_History,Property_Area,Income,Application_Status
0,LP001002,Male,No,0,Graduate,No,1,Urban,medium,Y
1,LP001003,Male,Yes,1,Graduate,No,1,Rural,medium,N
2,LP001005,Male,Yes,0,Graduate,Yes,1,Urban,low,Y
3,LP001006,Male,Yes,0,Not Graduate,No,1,Urban,low,Y
4,LP001008,Male,No,0,Graduate,No,1,Urban,medium,Y


# Data Preprocessing

In [115]:
data.shape

(511, 10)

In [114]:
cat_feature_x=[feature for feature in data.columns[:-1] if len(data[feature].unique())<10 and feature not in ['Application_ID']]
cat_feature_x
features=[feature for feature in data.columns[:-1] if feature not in ['Application_ID']]
x=data[features]
y=data.iloc[:,-1]

In [105]:
data.isnull().sum()

Application_ID        0
Gender                0
Married               0
Dependents            0
Education             0
Self_Employed         0
Credit_History        0
Property_Area         0
Income                0
Application_Status    0
dtype: int64

In [20]:
data.nunique()

Application_ID        511
Gender                  2
Married                 2
Dependents              4
Education               2
Self_Employed           2
Credit_History          2
Property_Area           3
Income                  3
Application_Status      2
dtype: int64

# Solution Q_1

In [38]:
s=data['Dependents'].unique()
for i in s:
    num_dep=data['Dependents'].value_counts()[i]
    per_dep=(num_dep/data.shape[0])*100
    print('Dependant unique value: '+i +' ;% of total applicants: ' + str(round(per_dep,2)) )

Dependant unique value: 0 ;% of total applicants: 57.53
Dependant unique value: 1 ;% of total applicants: 16.63
Dependant unique value: 2 ;% of total applicants: 17.22
Dependant unique value: 3+ ;% of total applicants: 8.61


# Solution Q_2

In [46]:
sum=0  # Here as the percentage of applicants with 3+ dependants are less,
       #hence I assume and approximate them as only having 3 dependants
for i in s:
    value=int(i[0])
    num_dep=data['Dependents'].value_counts()[i]
    per_dep=num_dep/data.shape[0]
    sum += per_dep*value
print('Average number of dependents per applicant: '+str(round(sum))+ '(Approx)')
   

Average number of dependents per applicant: 1.0(Approx)


# Solution Q_3

In [86]:
num_self_employed=0
approved=0
for i,value in enumerate(data['Self_Employed']):
    if value == 'Yes':
        num_self_employed += 1
        if data.loc[i, 'Application_Status'] == 'Y':
            approved += 1
per_approve=(approved/num_self_employed)*100
print('%of applications approved for self-employed applicants: '+ str(round(per_approve,2))) 

%of applications approved for self-employed applicants: 65.71


# Solution Q_4

In [140]:
rejection=0
total=0
for i in data.index:
    if data.loc[i, 'Gender'] == 'Male' and data.loc[i, 'Married'] == 'Yes':
        total += 1
        if data.loc[i, 'Application_Status'] == 'N':
            rejection += 1
per_rej=(rejection/total)*100
print('% of rejections for married male applicants: '+ str(round(per_rej,2)))        

% of rejections for married male applicants: 28.43


# Solution Q_5

In [142]:
data['Property_Area'].unique()

array(['Urban', 'Rural', 'Semiurban'], dtype=object)

In [158]:
approv_Ur=0
approv_r=0
approv_semi=0
ur=0
r=0
semi=0
for i in data.index:
    if data.loc[i, 'Property_Area'] == 'Urban':
        ur += 1
        if data.loc[i, 'Application_Status'] == 'Y':
            approv_Ur += 1
    elif data.loc[i, 'Property_Area'] == 'Rural':
        r += 1
        if data.loc[i, 'Application_Status'] == 'Y':
            approv_r += 1
    else:
        semi += 1
        if data.loc[i, 'Application_Status'] == 'Y':
            approv_semi += 1
            
lst={'Urban':approv_Ur/ur,'Rural':approv_r/r,'Semiurban':approv_semi/semi}
key_list = list(lst.keys())
val_list=list(lst.values())
print('property area with maximum approval ratio: '+ key_list[val_list.index(max(lst.values()))])


property area with maximum approval ratio: Semiurban


# Solution Q_6

In [159]:
data['Income'].unique()

array(['medium', 'low', 'high'], dtype=object)

In [169]:
dependant_low=[]
dependant_medium=[]
dependant_high=[]
for i in data.index:
    if data.loc[i,'Income'] == 'low':
        dependant_low.append(int(data.loc[i,'Dependents'][0]))
    elif data.loc[i,'Income'] == 'medium':
        dependant_medium.append(int(data.loc[i,'Dependents'][0]))
    else:
        dependant_high.append(int(data.loc[i,'Dependents'][0]))
        
print('Average Dependent for low income group: '+str(round(np.mean(dependant_low),1)))  
print('Average Dependent for medium income group: '+str(round(np.mean(dependant_medium),1)))  
print('Average Dependent for high income group: '+str(round(np.mean(dependant_high),1)))  

Average Dependent for low income group: 0.6
Average Dependent for medium income group: 0.9
Average Dependent for high income group: 0.9


# Solution Q_7

## One-hot encoding for catagorical data

In [133]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
col_trans=make_column_transformer((OneHotEncoder(),cat_feature_x),remainder='passthrough')
col_trans.fit_transform(x)

array([[0., 1., 1., ..., 0., 0., 1.],
       [0., 1., 0., ..., 0., 0., 1.],
       [0., 1., 0., ..., 0., 1., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 1.],
       [0., 1., 0., ..., 0., 0., 1.],
       [1., 0., 1., ..., 0., 0., 1.]])

In [128]:
from sklearn.pipeline import make_pipeline
from sklearn import svm #SVM classifier
clf = svm.SVC(kernel='rbf') # RBF Kernel
from sklearn.model_selection import cross_val_score

In [130]:
pipe=make_pipeline(col_trans,clf)
score=cross_val_score(pipe,x,y,cv=5,scoring='accuracy').mean()
print('Accuracy after cross validation: ' +str(score))

Accuracy after cross validation: 0.8081857985912813
